# SNP analysis
After running GWAS on longevity, longevity with disease score covariances and all diseases, we run finemapping of the longevity with disease covariance.  
Finemapping is performed via PolyFun.  
Follow insuctions at: https://github.com/omerwe/polyfun  
Finemapping results file was saved in output directory: output/finemap/polyfun_agg_longevity.txt.gz


In [2]:
source(here::here("code/init.R"))
source(here::here("code/gwas.R"))
options(tgutil.cache=FALSE)
pvals <- get_gwas_pvals() %cache_df% here("output/all_pvals.tsv")
snps <- pvals %>% filter(longevity_disease_covar_pval <=  log10(5e-8)) 
finemap_res <- tgutil::fread(here("output/finemap/polyfun_agg_longevity.txt.gz")) %>% 
    separate(CREDIBLE_SET, c("chrom_locus", "start_locus", "end_locus", "CREDIBLE_SET")) %>% 
    unite(chrom_locus:end_locus, col = "locus") %>% 
    as_tibble()


Joining with `by = join_by(rsid, allele1, allele2)`
Joining with `by = join_by(rsid, allele1, allele2)`
Joining with `by = join_by(rsid, allele1, allele2)`
Joining with `by = join_by(rsid, allele1, allele2)`
Joining with `by = join_by(rsid, allele1, allele2)`
Joining with `by = join_by(rsid, allele1, allele2)`


annotate finemapped snps

In [4]:
snps_annot <- snps %>%
        left_join(finemap_res %>% mutate(chrom = paste0("chr", CHR), start = BP, allele1 = A1, allele2 = A2)) %cache_df%
        here("output/longevity_snps_finemapped.tsv")
snps_annot %>% 
    filter(is.na(PIP)) %>% 
    count(chrom)
snps_annot %>% 
    filter(is.na(PIP)) %>% 
    nrow()

Joining with `by = join_by(chrom, start, allele1, allele2)`


chrom,n
<chr>,<int>
chr1,2
chr14,1
chr16,1
chr18,1
chr2,1
chr3,1
chr5,2
chr7,1
chr8,1


[1] 19

Choose at least one locus from each ~1M region + high PIP loci:

In [5]:
regs <- snps_annot %>%
        gintervals.centers() %>%
        gintervals.expand(5e5) %>%
        gintervals.canonic()
snps_top_annot <- snps_annot %>% 
    add_count(locus, name = "n_locus") %>%     
    gintervals.neighbors1(regs) %>% 
    select(-dist) %>% 
    unite("region", chrom1, start1, end1) %>% 
    add_count(region, name = "n_region") %>% 
    arrange(region, desc(PIP)) %>% 
    group_by(region) %>% 
    mutate(i = 1:n()) %>% 
    ungroup() %>% 
    filter(i == 1 | PIP >= 0.5) %>% 
    select(-i)  

Compute number of significant SNPs 1MB around each top snp:

In [6]:
snps_1M <- snps_top_annot %>%
        mutate(orig_chrom = chrom, orig_start = start, orig_end = end) %>% 
        gintervals.centers() %>%
        gintervals.expand(5e5) %>%
        select(chrom, start, end, marker.ID, allele1, allele2, orig_chrom, orig_start, orig_end) %>%
        gintervals.neighbors1(snps_annot %>% select(chrom, start, end), maxneighbors = 1e6) %>%
        as_tibble() %>%
        filter(dist == 0) %>%
        count(chrom, start, end, marker.ID, allele1, allele2, orig_chrom, orig_start, orig_end, name = "n_1M") %>%
        mutate(n_1M = n_1M - 1) %>%  # remove the SNP itself
        select(-(chrom:end)) %>% 
        rename(chrom = orig_chrom, start = orig_start, end = orig_end)
        
snps_top_annot <- snps_top_annot %>% left_join(snps_1M)

Joining with `by = join_by(chrom, start, end, marker.ID, allele1, allele2)`


Add results of COX regression (see GWAS_parents_survival notebook):

In [7]:
parents_mother <- fread(here("output/cox_parents_survival_mother_gwas.tsv")) %>% 
    inner_join(snps_top_annot %>% select(chrom, start, end, rsid, marker.ID, allele1, allele2)) %>% 
    as_tibble()
parents_father <- fread(here("output/cox_parents_survival_father_gwas.tsv")) %>% 
    inner_join(snps_top_annot %>% select(chrom, start, end, rsid, marker.ID, allele1, allele2)) %>% 
    as_tibble()
parents_both <- fread(here("output/cox_parents_survival_both_gwas.tsv")) %>% 
    inner_join(snps_top_annot %>% select(chrom, start, end, rsid, marker.ID, allele1, allele2)) %>% 
    as_tibble()

Joining with `by = join_by(chrom, start, end, marker.ID, allele1, allele2,
rsid)`
Joining with `by = join_by(chrom, start, end, marker.ID, allele1, allele2,
rsid)`
Joining with `by = join_by(chrom, start, end, marker.ID, allele1, allele2,
rsid)`


In [8]:
snps_top_annot <- snps_top_annot %>% 
    left_join(parents_mother %>% 
                select(chrom, start, end, rsid, marker.ID, allele1, allele2, cox_mother_surv_pval = pval, cox_mother_surv_z = z, cox_mother_surv_stat = Stat)             
             ) %>% 
    left_join(parents_father %>% 
                select(chrom, start, end, rsid, marker.ID, allele1, allele2, cox_father_surv_pval = pval, cox_father_surv_z = z, cox_father_surv_stat = Stat)             
             ) %>% 
    left_join(parents_both %>% 
                select(chrom, start, end, rsid, marker.ID, allele1, allele2, cox_both_surv_pval = pval, cox_both_surv_z = z, cox_both_surv_stat = Stat)             
             ) 

Joining with `by = join_by(chrom, start, end, marker.ID, rsid, allele1,
allele2)`
Joining with `by = join_by(chrom, start, end, marker.ID, rsid, allele1,
allele2)`
Joining with `by = join_by(chrom, start, end, marker.ID, rsid, allele1,
allele2)`


In [9]:
fwrite(snps_top_annot, here("output/longevity_top_finemapped.csv"))

In [ ]:
### H^2 SNP
### With Diseases as covariates
Create ldsc format sumstats:

adding std.err column from gwas of longevity with disease as covariance

In [11]:
gwas_longevity_disease_covar <- readr::read_rds(here("output/gwas_longevity_age_sex_disease_covar_extended.rds"))
pvals <- pvals %>% left_join(gwas_longevity_disease_covar %>% select(marker.ID, allele1, allele2, std.err))
pvals %>%
        filter(chrom != "chrX") %>% 
        mutate(N = 328542, CHR = gsub("chr", "", chrom),  Z = longevity_disease_covar_beta / std.err) %>%
        select(CHR, BP = start, A1 = allele1, A2 = allele2, N, Z, P = longevity_disease_covar_pval, SNP = rsid) %>%
        fwrite(here("output/longevity_snps_ldsc.sumstats"), sep = " ", quote = FALSE)

Joining with `by = join_by(marker.ID, allele1, allele2)`


In [ ]:
at the terminal (polyfun conda):
​
./ldsc.py \
--out /home/nettam/projects/emr/ukbiobank/notebook/output/longevity_snps_ldsc.h2 \
--h2 /home/nettam/projects/emr/ukbiobank/notebook/output/longevity_snps_ldsc.sumstats \
--ref-ld-chr baselineLF2.2.UKB/baselineLF2.2.UKB. \
--w-ld-chr baselineLF2.2.UKB/weights.UKB. \
--not-M-5-50
Total Observed scale h2: 0.1008 (0.0143)

### Without Diseases as covariates
Create ldsc format sumstats: 

In [12]:
gwas_longevity <- readr::read_rds(here("output/gwas_longevity_age_sex_covar_extended.rds"))
pvals <- pvals %>% left_join(gwas_longevity %>% select(marker.ID, allele1, allele2, std.err_longevity = std.err))
pvals %>%
        filter(chrom != "chrX") %>% 
        mutate(N = 328542, CHR = gsub("chr", "", chrom),  Z = longevity_beta / std.err_longevity) %>%
        select(CHR, BP = start, A1 = allele1, A2 = allele2, N, Z, P = longevity_pval, SNP = rsid) %>%
        fwrite(here("output/longevity_snps_no_covar_ldsc.sumstats"), sep = " ", quote = FALSE)
     


Joining with `by = join_by(marker.ID, allele1, allele2)`


In [ ]:
at the terminal (polyfun conda):

./ldsc.py \
--out /home/nettam/projects/emr/ukbiobank/notebook/output/longevity_snps_no_covar_ldsc.h2 \
--h2 /home/nettam/projects/emr/ukbiobank/notebook/output/longevity_snps_no_covar_ldsc.sumstats \
--ref-ld-chr baselineLF2.2.UKB/baselineLF2.2.UKB. \
--w-ld-chr baselineLF2.2.UKB/weights.UKB. \
--not-M-5-50
Total Observed scale h2: 0.1641 (0.0139)